In [84]:
import numpy as np
from scipy.stats import norm

In [85]:
def d(sigma, S, K, r, t):
    d1 = 1 / (sigma * np.sqrt(t)) * ( np.log(S/K) + (r + sigma**2/2) * t)
    d2 = d1 - sigma * np.sqrt(t)
    return d1, d2

In [86]:
def call_price(sigma, S, K, r, t, d1, d2):
    C = norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r * t)
    return C

In [87]:
def put_price(sigma, S, K, r, t, d1, d2):
    P = -norm.cdf(-d1) * S + norm.cdf(-d2) * K * np.exp(-r * t)
    return P

In [88]:
# delta 
def delta(d_1, contract_type):
    if contract_type == 'c':
        return norm.cdf(d1)
    if contract_type == 'p':
        return -norm.cdf(-d_1)

In [89]:
 #gamma  
def gamma(d2, S, K, sigma, r, t):
    return( K * np.exp(-r * t) * (norm.pdf(d2) / (S**2 * sigma * np.sqrt(t) ))) 


In [90]:
#theta 
def theta(d1, d2, S, K, sigma, r, t, contract_type):
    if contract_type == 'c':
        theta = -S * sigma * norm.pdf(d1) / (2 * np.sqrt(t)) - r * K * np.exp(-r * t) * norm.cdf(d2)
    if contract_type == 'p':
        theta = -S * sigma * norm.pdf(-d1) / (2 * np.sqrt(t)) + r * K * np.exp(-r * t) * norm.cdf(-d2)

    return theta

In [91]:
#vega 
def vega(sigma, S, K, r, t):
    d1, d2 = d(sigma, S, K, r, t)
    v = S * norm.pdf(d1) * np.sqrt(t)
    return v

In [92]:
#  Using ICICI data from 5/9/2020 closing values
S = 915; print('S = ', S)
K = 920; print('K = ', K)
r = 0.005; print('r = ', r)
t = 12 / 365; print('t = ', t)
sigma = 0.238; print('sigma = ', sigma)

S =  915
K =  920
r =  0.005
t =  0.03287671232876712
sigma =  0.238


In [93]:
#  Calculate the values of d1 and d2 needed for other functions
d1, d2 = d(sigma, S, K, r, t)
print('d1 = ', d1)
print('d2 = ', d2)


d1 =  -0.1008964650052079
d2 =  -0.14405047400959456


In [94]:
delta_call = delta(d1, 'c')
delta_put = delta(d1, 'p')
print('Call Delta = ', delta_call)
print('Put Delta = ', delta_put)

Call Delta =  0.4598163246531495
Put Delta =  -0.5401836753468505


In [95]:
#####  Calculate Theta
print(S, K, r, t, sigma, d1, d2)
print( theta(d1, d2, S, K, sigma, r, t, 'c') )

print( 'Call Theta = ', theta(d1, d2, S, K, sigma, r, t, 'c') / 365 * 100)
print( 'Put Thata = ', theta(d1, d2, S, K, sigma, r, t, 'p') / 365 * 100)

915 920 0.005 0.03287671232876712 0.238 -0.1008964650052079 -0.14405047400959456
-240.39072191907542
Call Theta =  -65.86047175865079
Put Thata =  -64.60040493734573


In [96]:
gamma = gamma(d2, S, K, sigma, r, t)
print( 'Gamma = ', gamma )

Gamma =  0.010052110268061689


In [97]:
vega = vega(sigma, S, K, r, t) /100
print( 'vega =', vega )

vega = 0.6585136335006087


In [98]:
#equity which we need to sell 
# formula = lot * reamaing delta
LotNumber = 1375
interstict_Delta = 1 - delta_call*2
sell_equity= interstict_Delta * LotNumber
print('sell equity= ',sell_equity)

sell equity=  110.50510720383893


In [99]:
# sell equity if we show 1$ change in delta
#formula = sell_equity / (1- ((interstict value* gamma) + delta)) 
sell_as_per_gamma = sell_equity/(50-(((K - S) * gamma) + (delta_call*100)))
print("sell as per gamma = ", sell_as_per_gamma)

sell as per gamma =  27.848318522575056


In [100]:
# money we got on the call sell position if the primum will remain same (theta money)
#formula = theta * lot * number of the call/put 
call_theta = theta(d1, d2, S, K, sigma, r, t, 'c') / 365
print('money on theta = ',LotNumber * 2 * call_theta)


money on theta =  -1811.162973362897


In [101]:
# money we got on our hedge position if the primum will reamin same (vega)
#formula = vega * lot * number of the call/put
print('money on vega= ', LotNumber * 2 * vega)

money on vega=  1810.912492126674
